# ISYE/Math/CS 425 - Assignment 4
### Bryan Luu

## Exercise 5

Max-flow instance set-up

In [186]:
P_str = "abcdefghij"; # Specify P here as a string
Q_str = uppercase(P_str); # Specify Q here as a string

to_sym(v) = map(x->Symbol(x), v); # function to convert string into array of symbols
P, Q = to_sym(collect(P_str)), to_sym(collect(Q_str)); # creates arrays for P and Q

V = Set([P..., Q..., :r, :s]); # Node Set

L = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Adjacency list
L[:a] = [:A, :C, :D];
L[:b] = [:A, :B, :H];
L[:c] = [:A, :C];
L[:d] = [:C, :D];
L[:e] = [:C, :D];
L[:f] = [:B, :E, :G];
L[:g] = [:E, :F, :J];
L[:h] = [:E, :F, :G, :H, :J];
L[:i] = [:I];
L[:j] = [:D, :I];

L[:r] = P;
for v in Q
    L[v] = [:s];
end

E = Set(); # Edge Set
for i in keys(L)
    for j in L[i]
        push!(E, (i, j));
    end
end

x = Dict(zip(E, zeros(length(E)))); # Flows
u = Dict(zip(E, Inf*ones(length(E)))); # Capacities
for p in P
    u[(:r, p)] = 1;
end
for q in Q
    u[(q, :s)] = 1;
end

In [187]:
function flow(v)
    inflow = sum(Float64[x[(i,j)] for (i,j) in E if j==v]);
    outflow = sum(Float64[x[(i,j)] for (i,j) in E if i==v]);
    inflow - outflow
end

flow (generic function with 1 method)

In [188]:
# Bread-first-search for (u,v)-path in the graph defined by adjacency list L
function BFS(L, u, v)
    p = Dict{Symbol, Any}(zip(V, -1*ones(length(V))));
    p[u] = 0;
    q = [u]; # queue for BFS
    while !isempty(q)
        n = popfirst!(q);
        if n == v
            break
        end
        for m in L[n]
            if p[m] == -1
                push!(q, m);
                p[m] = n;
            end
        end
    end
    i = v;
    path = [];
    while(i != u)
        if p[i] == -1
            return []
        end
        pushfirst!(path, i);
        i = p[i];
    end
    pushfirst!(path, u);
    path
end

BFS (generic function with 2 methods)

In [189]:
# Augmenting path algorithm

path = BFS(L, :r, :s);
i = 1;
while(!isempty(path))
    println("---Iteration $(i)---")
    # find x-width
    arcs = [p for p in zip(path[1:end-1], path[2:end])];
    eps1 = min([u[e] - x[e] for e in arcs if e in E]...);
    eps2 = min(Inf, [x[e] for e in arcs if reverse(e) in E]...);
    x_width = min(eps1, eps2);
    println(path)
    println("e\tx[e]")
                                    
    if x_width == Inf
        break
    end
                                    
    for e in arcs
        if e in E
            x[e] += x_width; # increase flow on forward arcs
        else
            x[reverse(e)] -= x_width; # decrease flow on reverse arcs
        end
    end
    
    for e in E
        println("$(e)\t$(x[e])")
    end
                                    
    # construct augmented graph
    La = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Adjacency list
    for (v, w) in E
        if x[(v, w)] < u[(v, w)]
            push!(La[v], w);
        end
        if x[(v, w)] > 0
            push!(La[w], v);
        end
    end
    
    println("G':\n$(join(["$v\t$(La[v])" for v in V], "\n"))")
                                    
    path = BFS(La, :r, :s);
    i += 1;
end

---Iteration 1---
Any[:r, :a, :A, :s]
e        	x[e]
(:r, :b)	0.0
(:r, :d)	0.0
(:b, :B)	0.0
(:e, :C)	0.0
(:c, :C)	0.0
(:h, :H)	0.0
(:r, :i)	0.0
(:g, :F)	0.0
(:h, :F)	0.0
(:a, :D)	0.0
(:g, :J)	0.0
(:h, :E)	0.0
(:h, :J)	0.0
(:g, :E)	0.0
(:h, :G)	0.0
(:b, :A)	0.0
(:r, :j)	0.0
(:d, :C)	0.0
(:r, :a)	1.0
(:G, :s)	0.0
(:A, :s)	1.0
(:B, :s)	0.0
(:C, :s)	0.0
(:e, :D)	0.0
(:H, :s)	0.0
(:f, :E)	0.0
(:f, :G)	0.0
(:F, :s)	0.0
(:D, :s)	0.0
(:E, :s)	0.0
(:c, :A)	0.0
(:f, :B)	0.0
(:J, :s)	0.0
(:a, :C)	0.0
(:r, :h)	0.0
(:i, :I)	0.0
(:b, :H)	0.0
(:r, :g)	0.0
(:j, :I)	0.0
(:a, :A)	1.0
(:r, :c)	0.0
(:r, :e)	0.0
(:j, :D)	0.0
(:r, :f)	0.0
(:d, :D)	0.0
(:I, :s)	0.0
G':
j	Any[:I, :D]
D	Any[:s]
d	Any[:C, :D]
A	Any[:a]
g	Any[:F, :J, :E]
r	Any[:b, :d, :i, :j, :h, :g, :c, :e, :f]
a	Any[:D, :r, :C, :A]
J	Any[:s]
i	Any[:I]
b	Any[:B, :A, :H]
F	Any[:s]
e	Any[:C, :D]
B	Any[:s]
c	Any[:C, :A]
h	Any[:H, :F, :E, :J, :G]
E	Any[:s]
s	Any[:A]
f	Any[:E, :G, :B]
I	Any[:s]
H	Any[:s]
G	Any[:s]
C	Any[:s]
---Iteration 2---
Any[:r,

In [201]:
# x is now the max-flow
println("Max-flow: $(flow(:s))")
M = [(p, q) for (p, q) in keys(x) if x[(p, q)] == 1 && (in(p, P) || in(q, Q))];
println("Matchings:")
println(join(sort(M), "\n"))

Max-flow: 8.0
Matchings:
(:a, :A)
(:b, :B)
(:d, :C)
(:f, :E)
(:g, :F)
(:h, :H)
(:i, :I)
(:j, :D)


true

9-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7
 8
 9